In [1]:
import pickle
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar

In [2]:
lstm_result_path = "../pickle/results_latefee.pickle"
classifiers_result_path = "../pickle/classifiers_allclients.pickle"

In [3]:
def compare(first_dict, second_dict):
    dictionary = {}
    for i in first_dict.keys():
        first = first_dict[i]
        second = second_dict[i]
        res = first.merge(second, on='Key', how='left')
        res[['scores_x', 'scores_y']] = res[['scores_x', 'scores_y']].round(decimals=0)

        yes_yes = res[(res['actual_x'] != res['scores_x']) & (res['actual_x'] != res['scores_y'])].shape[0]
        yes_no = res[(res['actual_x'] != res['scores_x']) & (res['actual_x'] == res['scores_y'])].shape[0]
        no_yes = res[(res['actual_x'] == res['scores_x']) & (res['actual_x'] != res['scores_y'])].shape[0]
        no_no = res[(res['actual_x'] == res['scores_x']) & (res['actual_x'] == res['scores_y'])].shape[0]

        table = np.array([[yes_yes, yes_no], [no_yes, no_no]])
        test = mcnemar(table, exact=False)
        dictionary[i] = dict(pvalue=test.pvalue, statistic=test.statistic)
    return dictionary

In [4]:
with open(lstm_result_path, "rb") as fp:
    lstm_dict = pickle.load(fp)
with open(classifiers_result_path, "rb") as fp:
    classifiers_dict = pickle.load(fp)
result = {}
for i in classifiers_dict.keys():
    result['LSTM vc ' + str(i)] = compare(lstm_dict, classifiers_dict[i])

with open("../pickle/McNemar.pickle", "wb") as fp:
    pickle.dump(result, fp)

print(result)

{'LSTM vc Neural Network': {'train': {'pvalue': 0.0, 'statistic': 2788.8375812743825}, 'test': {'pvalue': 2.7039565695768675e-26, 'statistic': 112.55259822560203}}, 'LSTM vc SVM': {'train': {'pvalue': 0.0, 'statistic': 2820.70206914615}, 'test': {'pvalue': 9.978541548314505e-64, 'statistic': 284.02240143369175}}, 'LSTM vc Random Forest': {'train': {'pvalue': 0.0, 'statistic': 4072.422804620544}, 'test': {'pvalue': 1.5869799369232135e-47, 'statistic': 209.71258671952427}}, 'LSTM vc Logistic Regression': {'train': {'pvalue': 0.0, 'statistic': 2671.8223529411766}, 'test': {'pvalue': 1.7364736012297305e-56, 'statistic': 250.80167597765364}}}
